# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [5]:
import numpy as np 
import pandas as pd
import requests
import xlsxwriter
from scipy import stats
import math

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [6]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks = stocks[~stocks['Ticker'].isin(['DISCA', 'HFC','VIAC','WLTW'])]
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [9]:
symbol = 'aapl'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'avgTotalVolume': 89693268, 'calculationPrice': 'close', 'change': 3.44, 'changePercent': 0.02573, 'close': 147, 'closeSource': 'oficfila', 'closeTime': 1679368629761, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': 142.04, 'delayedPriceTime': 1683744413378, 'extendedChange': 0.08, 'extendedChangePercent': 0.00057, 'extendedPrice': 142.97, 'extendedPriceTime': 1736483068096, 'high': 147, 'highSource': 'm yiutl1nede5 erca iped', 'highTime': 1672874493093, 'iexAskPrice': None, 'iexAskSize': None, 'iexBidPrice': None, 'iexBidSize': None, 'iexClose': 148.012, 'iexCloseTime': 1691604140221, 'iexLastUpdated': None, 'iexMarketPercent': None, 'iexOpen': 144.18, 'iexOpenTime': 1667219625201, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexVolume': None, 'lastTradeTime': 1656817944614, 'latestPrice': 144.94, 'latestSource': 'Close', 'latestTime': 'June 24, 2022', 'latestUpdate': 1660830458120, 'latestVolume': 93557162, 'low': 139.92, 'lowSource': 'l edti1epmc eu ydeia5rn',

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [12]:
price = data['latestPrice']
pe_ratio = data['peRatio']

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [13]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [32]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
        pd.Series(
        [
            symbol,
            data[symbol]['quote']['latestPrice'],
            data[symbol]['quote']['peRatio'],
            'N/A'
        ], index = my_columns),
        ignore_index = True
        )
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,126.37,30.09,N/A
1,AAL,14.40,-3.96,N/A
2,AAP,187.20,20.5,N/A
3,AAPL,145.77,23.29,N/A
4,ABBV,154.39,22.52,N/A
...,...,...,...,...
496,YUM,114.91,20.9,N/A
497,ZBH,111.07,107.92,N/A
498,ZBRA,320.90,20.42,N/A
499,ZION,55.36,5.73,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [34]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True, drop = True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,39.89,2.36,N/A
1,EBAY,46.48,2.5,N/A
2,AMZN,116.70,2.9,N/A
3,SYF,30.56,4.02,N/A
4,NUE,115.38,4.19,N/A
5,F,12.46,4.36,N/A
6,COF,115.57,4.46,N/A
7,PVH,65.02,4.6,N/A
8,AIG,53.49,4.77,N/A
9,DHI,70.42,4.92,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [35]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [36]:
portfolio_input()

Enter the value of your portfolio:one million dollars
That's not a number! 
 Try again:
Enter the value of your portfolio:1000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [41]:
position_size = float(portfolio_size) / len(final_dataframe.index)

for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[row, 'Price'])
    
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,39.89,2.36,501
1,EBAY,46.48,2.5,430
2,AMZN,116.70,2.9,171
3,SYF,30.56,4.02,654
4,NUE,115.38,4.19,173
5,F,12.46,4.36,1605
6,COF,115.57,4.46,173
7,PVH,65.02,4.6,307
8,AIG,53.49,4.77,373
9,DHI,70.42,4.92,284


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [48]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote,advanced-stats&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# Price-to_earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']

# Price-to-book ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

# Price-to-sales ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value / ebitda

# Enterprise Value divided by Gross Profit (EV/GP)
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value / gross_profit

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [57]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)


for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote,advanced-stats&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        try:
            ev_to_gp = enterprise_value / gross_profit
        except TypeError:
            ev_to_gp = np.NaN
        rv_dataframe = rv_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gp,
                'N/A',
                'N/A'
            ], index = rv_columns
            ), ignore_index = True
        )
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,126.08,N/A,29.32,N/A,7.07,N/A,5.74,N/A,21.666546,N/A,10.653551,N/A,N/A
1,AAL,14.00,N/A,-3.84,N/A,-1.04,N/A,0.2703,N/A,-21.725653,N/A,0.975745,N/A,N/A
2,AAP,186.90,N/A,20.42,N/A,4.0,N/A,1.06,N/A,11.361539,N/A,2.513037,N/A,N/A
3,AAPL,142.87,N/A,23.3,N/A,34.68,N/A,6.18,N/A,17.663450,N/A,13.833573,N/A,N/A
4,ABBV,159.71,N/A,21.84,N/A,17.02,N/A,4.77,N/A,11.488840,N/A,8.468584,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,117.62,N/A,21.65,N/A,-3.98,N/A,5.11,N/A,19.509389,N/A,8.914219,N/A,N/A
497,ZBH,113.67,N/A,110.17,N/A,1.9,N/A,3.11,N/A,17.985268,N/A,5.481421,N/A,N/A
498,ZBRA,319.83,N/A,20.32,N/A,5.82,N/A,2.93,N/A,16.542063,N/A,6.728348,N/A,N/A
499,ZION,54.93,N/A,5.56,N/A,1.41,N/A,2.42,N/A,4.492924,N/A,2.218342,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [58]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
28,ALXN,191.600,N/A,61.38,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,282.330,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,292.180,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
88,CERN,98.790,N/A,49.42,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,10.04,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
135,DISCK,25.360,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
164,ETFC,51.420,N/A,14.35,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
185,FLIR,58.460,N/A,32.43,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
189,FOX,31.150,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
192,FRT,102.140,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [63]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio', 'Price-to-Sales Ratio', 'EV/EBITDA', 'EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [64]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [70]:
from scipy.stats import percentileofscore as score
metrics = { 'Price-to-Earnings Ratio':'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio':'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'}

for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = score( rv_dataframe[metric], rv_dataframe.loc[row, metric])/100
        
rv_dataframe        

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,126.08,N/A,29.32,0.710579,7.07,0.814371,5.7400,0.826347,21.666546,0.820359,10.653551,0.792415,N/A
1,AAL,14.00,N/A,-3.84,0.033932,-1.04,0.05988,0.2703,0.013972,-21.725653,0.007984,0.975745,0.03992,N/A
2,AAP,186.90,N/A,20.42,0.489022,4.00,0.61976,1.0600,0.163673,11.361539,0.373253,2.513037,0.149701,N/A
3,AAPL,142.87,N/A,23.30,0.58483,34.68,0.97006,6.1800,0.841317,17.663450,0.698603,13.833573,0.888224,N/A
4,ABBV,159.71,N/A,21.84,0.528942,17.02,0.932136,4.7700,0.758483,11.488840,0.377246,8.468584,0.672655,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,117.62,N/A,21.65,0.522954,-3.98,0.055888,5.1100,0.789421,19.509389,0.762475,8.914219,0.708583,N/A
497,ZBH,113.67,N/A,110.17,0.984032,1.90,0.278443,3.1100,0.553892,17.985268,0.714571,5.481421,0.401198,N/A
498,ZBRA,319.83,N/A,20.32,0.487026,5.82,0.748503,2.9300,0.528942,16.542063,0.662675,6.728348,0.508982,N/A
499,ZION,54.93,N/A,5.56,0.07984,1.41,0.179641,2.4200,0.44012,4.492924,0.075848,2.218342,0.133733,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [71]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,126.08,N/A,29.32,0.710579,7.07,0.814371,5.7400,0.826347,21.666546,0.820359,10.653551,0.792415,0.792814
1,AAL,14.00,N/A,-3.84,0.033932,-1.04,0.05988,0.2703,0.013972,-21.725653,0.007984,0.975745,0.03992,0.031138
2,AAP,186.90,N/A,20.42,0.489022,4.00,0.61976,1.0600,0.163673,11.361539,0.373253,2.513037,0.149701,0.359082
3,AAPL,142.87,N/A,23.30,0.58483,34.68,0.97006,6.1800,0.841317,17.663450,0.698603,13.833573,0.888224,0.796607
4,ABBV,159.71,N/A,21.84,0.528942,17.02,0.932136,4.7700,0.758483,11.488840,0.377246,8.468584,0.672655,0.653892
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,117.62,N/A,21.65,0.522954,-3.98,0.055888,5.1100,0.789421,19.509389,0.762475,8.914219,0.708583,0.567864
497,ZBH,113.67,N/A,110.17,0.984032,1.90,0.278443,3.1100,0.553892,17.985268,0.714571,5.481421,0.401198,0.586427
498,ZBRA,319.83,N/A,20.32,0.487026,5.82,0.748503,2.9300,0.528942,16.542063,0.662675,6.728348,0.508982,0.587226
499,ZION,54.93,N/A,5.56,0.07984,1.41,0.179641,2.4200,0.44012,4.492924,0.075848,2.218342,0.133733,0.181836


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [73]:
rv_dataframe.sort_values('RV Score', inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AAL,14.00,N/A,-3.84,0.033932,-1.0400,0.05988,0.2703,0.013972,-21.725653,0.007984,0.975745,0.03992,0.031138
1,AIG,52.78,N/A,4.80,0.061876,0.7661,0.075848,0.8048,0.115768,2.744922,0.02994,0.759910,0.017964,0.060279
2,GPS,9.71,N/A,-52.33,0.00998,1.4400,0.186627,0.2162,0.007984,5.075088,0.097804,0.723627,0.013972,0.063273
3,GM,35.58,N/A,6.03,0.093812,0.8255,0.081836,0.4019,0.033932,1.950566,0.01996,1.626327,0.091816,0.064271
4,UNM,34.31,N/A,7.89,0.127745,0.6726,0.071856,0.5844,0.067864,3.903190,0.051896,0.577105,0.007984,0.065469
5,LNC,49.05,N/A,7.25,0.111776,0.5813,0.065868,0.4510,0.03992,5.244553,0.10978,0.439142,0.001996,0.065868
6,PVH,63.89,N/A,4.61,0.05988,0.8159,0.07984,0.4828,0.0499,4.753802,0.091816,1.080254,0.051896,0.066667
7,F,12.36,N/A,4.32,0.055888,1.1000,0.116766,0.3541,0.025948,3.695333,0.043912,1.889997,0.105788,0.069661
8,KSS,40.03,N/A,5.93,0.08982,1.1600,0.129741,0.2696,0.011976,4.031062,0.057884,1.149010,0.05988,0.06986
9,SYF,29.87,N/A,4.05,0.052894,1.1700,0.133733,0.8969,0.127745,2.485844,0.023952,0.930391,0.027944,0.073253


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [74]:
portfolio_input()

Enter the value of your portfolio:1000000


In [76]:
position_size = float(portfolio_size) / len(rv_dataframe.index)

for row in rv_dataframe.index:
    rv_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe.loc[row, 'Price'])
    
rv_dataframe

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score,Numer of Shares to Buy
0,AAL,14.00,1428,-3.84,0.033932,-1.0400,0.05988,0.2703,0.013972,-21.725653,0.007984,0.975745,0.03992,0.031138,1428.0
1,AIG,52.78,378,4.80,0.061876,0.7661,0.075848,0.8048,0.115768,2.744922,0.02994,0.759910,0.017964,0.060279,378.0
2,GPS,9.71,2059,-52.33,0.00998,1.4400,0.186627,0.2162,0.007984,5.075088,0.097804,0.723627,0.013972,0.063273,2059.0
3,GM,35.58,562,6.03,0.093812,0.8255,0.081836,0.4019,0.033932,1.950566,0.01996,1.626327,0.091816,0.064271,562.0
4,UNM,34.31,582,7.89,0.127745,0.6726,0.071856,0.5844,0.067864,3.903190,0.051896,0.577105,0.007984,0.065469,582.0
5,LNC,49.05,407,7.25,0.111776,0.5813,0.065868,0.4510,0.03992,5.244553,0.10978,0.439142,0.001996,0.065868,407.0
6,PVH,63.89,313,4.61,0.05988,0.8159,0.07984,0.4828,0.0499,4.753802,0.091816,1.080254,0.051896,0.066667,313.0
7,F,12.36,1618,4.32,0.055888,1.1000,0.116766,0.3541,0.025948,3.695333,0.043912,1.889997,0.105788,0.069661,1618.0
8,KSS,40.03,499,5.93,0.08982,1.1600,0.129741,0.2696,0.011976,4.031062,0.057884,1.149010,0.05988,0.06986,499.0
9,SYF,29.87,669,4.05,0.052894,1.1700,0.133733,0.8969,0.127745,2.485844,0.023952,0.930391,0.027944,0.073253,669.0


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [81]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine = 'xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name = 'Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [84]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0.0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [86]:
column_formats = {
                    'A':['Ticker', string_template],
                    'B':['Price', dollar_template],
                    'C':['Number of Shares to Buy', integer_template],
                    'D':['Price-to-Earnings Ratio', float_template],
                    'E':['PE Percentile', percent_template],
                    'F':['Price-to-Book Ratio', float_template],
                    'G':['PB Percentile', percent_template],
                    'H':['Price-to-Sales Ratio', float_template],
                    'I':['PS Percentile', percent_template],
                    'J':['EV/EBITDA', float_template],
                    'K':['EV/EBITDA Percentile', percent_template],
                    'L':['EV/GP', float_template],
                    'M':['EV/GP Percentile', percent_template],
                    'N':['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [88]:
writer.save()